In [ ]:
import pandas as pd
from pandas.tseries.offsets import DateOffset
from fredapi import Fred

In [ ]:
# Note: Prior to running Program, ensure that the stock_data.csv is in the same jupyter notebook

In [ ]:
# establish time frame from stock data (this will work for most time frames)
data = pd.read_csv("stock_data.csv", parse_dates =["Date"]) # NOTE: Update to your file's date/time column name if needed
data.set_index("Date", inplace=True) # NOTE: Update to your file's date/time collumn name if needed
data = data.sort_index(ascending=True) # Set to decreasing order
data.index = pd.to_datetime(data.index, utc=True)
data_index = data.index # create index object for use in macro dataframe below
data_index

In [ ]:
# Establish Start and End Date for Fred Data Pull 
start_date = data_index[0] - DateOffset(years=1) # Pull additional data (we will discard excess later)
end_date = None
start_date=pd.to_datetime(start_date,utc=True)

In [ ]:
# initialize Fred API
# Note: To use FRED API you must register an account and receive an api_key
fred = Fred(api_key = 'a1e14ac266190134fdbc578b6966ef59')

In [ ]:
# GDP Historical Pull and Format
gdp_data = fred.get_series("GDPC1", observation_start = start_date, observation_end = end_date)
gdp_df = pd.DataFrame(gdp_data)
gdp_df.columns = ["GDP"]
gdp_df.index.name = "date"
gdp_df["GDP QoQ"] = gdp_df["GDP"].pct_change() * 100
gdp_df["GDP YoY"] = gdp_df["GDP"].pct_change(4) * 100
gdp_df = gdp_df.dropna()
gdp_df.drop('GDP', axis=1, inplace=True)
gdp_df.index = pd.to_datetime(gdp_df.index, utc=True)
gdp_df

In [ ]:
# CPI Historical Pull and Format (We use Median Consumer Price Index)
cpi_data = fred.get_series("MEDCPIM158SFRBCLE", observation_start = start_date, observation_end=end_date)
cpi_df = pd.DataFrame(cpi_data)
cpi_df.columns = ["CPI"]
cpi_df.index.name = "date"
cpi_df.index = pd.to_datetime(cpi_df.index, utc=True)
cpi_df

In [ ]:
# Fed Fund Historical Pull and Format
ff_data = fred.get_series("FEDFUNDS", observation_start = start_date, observation_end = end_date)
ff_df = pd.DataFrame(ff_data)
ff_df.columns = ["FEDFUNDS"]
ff_df.index.name = "date"
ff_df.index = pd.to_datetime(ff_df.index, utc=True)
ff_df

In [ ]:
# Treasury (2Y, 10Y, 2-10 Spread) Historical Pull and Format
tsy_ids = ['DGS10', 'DGS2']
data_dict = {series_id : fred.get_series(series_id, observation_start = start_date, observation_end = end_date) 
             for series_id in tsy_ids}
tsy_df = pd.DataFrame(data_dict)
tsy_df.columns = ["TSY 10YY", "TSY 2YY"]
tsy_df.index.name = "date"
tsy_df.index = pd.to_datetime(tsy_df.index, utc=True)
tsy_df["TSY 2-10 Spread"] = tsy_df["TSY 10YY"] - tsy_df["TSY 2YY"]
tsy_df

In [ ]:
# Create Macro Data that fits time-frame of the stock data (using ffill where appropriate)
macro_data = pd.DataFrame(index=data_index) # use stock_data index for indexing of macro data
macro_data["TSY 2YY"] = tsy_df.reindex(macro_data.index, method = 'ffill')["TSY 2YY"]
macro_data["TSY 10YY"] = tsy_df.reindex(macro_data.index, method = 'ffill')["TSY 10YY"]
macro_data["TSY 2-10 Spread"] = tsy_df.reindex(macro_data.index, method = 'ffill')["TSY 2-10 Spread"]
macro_data["GDP QoQ"] = gdp_df.reindex(macro_data.index, method = 'ffill')["GDP QoQ"]
macro_data["GDP YoY"] = gdp_df.reindex(macro_data.index, method = 'ffill')["GDP YoY"]
macro_data["FEDFUNDS"] = ff_df.reindex(macro_data.index, method = 'ffill')["FEDFUNDS"]
macro_data["CPI"] = cpi_df.reindex(macro_data.index, method = 'ffill')["CPI"]
macro_data

In [ ]:
# Incorporate Macro Data into Stock DataFrame
data["TSY 2YY"] = macro_data["TSY 2YY"]
data["TSY 10YY"] = macro_data["TSY 10YY"]
data["TSY 2-10 Spread"] = macro_data["TSY 2-10 Spread"]
data["GDP QoQ"] = macro_data["GDP QoQ"]
data["GDP YoY"] = macro_data["GDP YoY"]
data["FEDFUNDS"] = macro_data["FEDFUNDS"]
data["CPI"] = macro_data["CPI"]
data

In [ ]:
# Export Main DataFrame with Stock and Macro Data (Update with your own filename if desired)
data.to_csv("ML_data.csv") 

In [ ]:
# Export Macro only-data if desored
macro_data.to_csv("macro_data.csv")